<a href="https://colab.research.google.com/github/amoghagastya/proven-club-mle-challenge-1/blob/main/synthesize_summaries_finetune_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: read dataset using pandas

import pandas as pd

# Read a dataset from a CSV file
df = pd.read_csv('/content/tpc-challenge-2/Dataset problem 3.csv')

# Print the first few rows of the dataset
df.head()


,abstract
0,This clinical trial investigates the long-term...
1,The objective of this research is to determine...
2,The primary outcome of this trial is to measur...
3,This research focuses on the pharmacokinetics ...
4,A randomized controlled trial to assess the im...


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0, api_key='')

In [ ]:
from langchain_core.prompts import PromptTemplate
template = """Your task is to generate concise summaries of medical research abstracts.
Here is the abstract: {abstract}

Concise Summary:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm

question = "This clinical trial investigates the long-term effects of medication Z on cardiovascular health. A double-blind study to compare the effectiveness of treatment A versus placebo in patients with condition B. The objective of this research is to determine the safety and tolerability of the combination therapy for cancer patients. A randomized controlled trial to assess the impact of intervention Y on the recovery time of patients after surgery. The primary outcome of this trial is to measure the reduction in symptoms of patients with chronic illness C after using the new treatment."

chain.invoke(question)


AIMessage(content="1. Study on medication Z's long-term effects on cardiovascular health.\n2. Comparison of treatment A versus placebo for patients with condition B.\n3. Safety and tolerability of combination therapy for cancer patients.\n4. Impact of intervention Y on post-surgery recovery time.\n5. Reduction in symptoms of chronic illness C with new treatment.", response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 130, 'total_tokens': 197}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-05e7e356-b6c5-4bb1-94b4-1a3797a1f535-0')

In [ ]:
# prompt: for every row in the dataset, use chain.invoke and then add the result.content to a new column in the dataset called summary

df['summary'] = df['Abstract'].apply(lambda x: chain.invoke(x).content)
df.head()


,abstract,summary
0,This clinical trial investigates the long-term...,1. Study on medication Z's long-term effects o...
1,The objective of this research is to determine...,1. Research aims to assess safety and tolerabi...
2,The primary outcome of this trial is to measur...,1. Trial measures reduction in symptoms of chr...
3,This research focuses on the pharmacokinetics ...,This research examines the effects of an inves...
4,A randomized controlled trial to assess the im...,1. Intervention Y was tested in a randomized c...


In [ ]:
# prompt: Using dataframe df: make the summary column equal to rows in df

df['summary'] = df.apply(lambda row: row['abstract'], axis=1)


In [ ]:
from tqdm import tqdm

df['summary'] = df['abstract']

for index, row in tqdm(df.iterrows()):
    summary = chain.invoke(row['abstract']).content
    df.loc[index, 'summary'] = summary

df.head()

1000it [21:47,  1.31s/it]


,abstract,summary
0,This clinical trial investigates the long-term...,1. Study on medication Z's long-term effects o...
1,The objective of this research is to determine...,This research aims to assess the safety and to...
2,The primary outcome of this trial is to measur...,1. Trial measures reduction in symptoms of chr...
3,This research focuses on the pharmacokinetics ...,This research examines the effects of an inves...
4,A randomized controlled trial to assess the im...,1. Intervention Y was tested in a randomized c...


In [ ]:
df = pd.read_csv('/content/tpc-challenge-2/medical-abstract-summaries.csv')

In [ ]:
df.head()

,Unnamed: 0,abstract,summary
0,0,This clinical trial investigates the long-term...,1. Study on medication Z's long-term effects o...
1,1,The objective of this research is to determine...,This research aims to assess the safety and to...
2,2,The primary outcome of this trial is to measur...,1. Trial measures reduction in symptoms of chr...
3,3,This research focuses on the pharmacokinetics ...,This research examines the effects of an inves...
4,4,A randomized controlled trial to assess the im...,1. Intervention Y was tested in a randomized c...


In [ ]:
# prompt: split this datset into train and test... 950 rows for training and rest for validate

train_df = df.iloc[:950]
test_df = df.iloc[950:]

print("Train dataset size:", train_df.shape)
print("Test dataset size:", test_df.shape)


Train dataset size: (950, 3)
Test dataset size: (50, 3)


In [ ]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)